# **Automate Documents Categorization- PyTorch & Neural Networks**


The implementation of an automated machine learning system makes it very efficient. Such a system, equipped with advanced natural language processing and machine learning capabilities, could sift through the vast archives, categorizing articles into their respective topics with remarkable precision. As a result, readers would seamlessly access a wealth of knowledge tailored to their interests, while the editorial team gains newfound agility in content management.



# Setup


### Installing Required Libraries



In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# - Update a specific package
# !pip install pmdarima -U
# - Update a package to specific version
# !pip install --upgrade pmdarima==2.0.2
# Note: If your environment doesn't support "!pip install", use "!mamba install"

In [ ]:
!pip install -Uqq portalocker>=2.0.0
!pip install -qq torchtext
!pip install -qq torchdata
!pip install -Uqq plotly

### Importing Required Libraries


In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
import numpy as np
from torchtext.datasets import AG_NEWS
from IPython.display import Markdown as md
from tqdm import tqdm

from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from sklearn.manifold import TSNE
import plotly.graph_objs as go


def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
def plot(COST,ACC):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('epoch', color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis='y', color=color)
    
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)  # we already handled the x-label with ax1
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    
    plt.show()

# Toy Dataset 
To gain a deeper understanding of the TorchText pipeline, let's engage with a toy dataset through interactive exploration. 

We have a dataset that consists of a list of tuples, where each tuple contains a random numeric label and a text document.


In [ ]:
dataset = [
    (1,"Introduction to NLP"),
    (2,"Basics of PyTorch"),
    (1,"NLP Techniques for Text Classification"),
    (3,"Named Entity Recognition with PyTorch"),
    (3,"Sentiment Analysis using PyTorch"),
    (3,"Machine Translation with PyTorch"),
    (1," NLP Named Entity,Sentiment Analysis,Machine Translation "),
    (1," Machine Translation with NLP "),
    (1," Named Entity vs Sentiment Analysis  NLP ")]

### Tokenizer


First import the **```get_tokenizer```** function from **```torchtext.data.utils```**


In [ ]:
from torchtext.data.utils import get_tokenizer

Next, we create the tokenizer, we set it to "basic_english" tokenizer provided by torchtext. The "basic_english" tokenizer is designed to handle basic English text and splits text into individual tokens based on spaces and punctuation marks.


In [ ]:
tokenizer = get_tokenizer("basic_english")

We iterate over each tuple (y, sentence) in the corpus, tokenizing the sentence. Therefore, each word becomes a token for the computer to understand individual words from a sentence.


In [ ]:
for y, sentence in dataset:
    # Print the label (y) associated with the current sentence
    print("y:", y)

    # Print the sentence text
    print("sentence:", sentence)

    # Tokenize the sentence using the "basic_english" tokenizer
    # The tokenizer splits the sentence into individual tokens (words)
    tokens = tokenizer(sentence)

    # Print the list of tokens obtained from tokenizing the sentence
    print("tokenizer:", tokens)

### Token Indices


We need to represent words as numbers as NLP algorithms can process and manipulate numbers more efficiently and quickly than raw text. We use the function **```build_vocab_from_iterator```**, the output is typically referred to as 'token indices' or simply 'indices.' These indices represent the numeric representations of the tokens in the vocabulary.

The **```build_vocab_from_iterator```** function, when applied to a list of tokens, assigns a unique index to each token based on its position in the vocabulary. These indices serve as a way to represent the tokens in a numerical format that can be easily processed by machine learning models.

For example, given a vocabulary with tokens ["apple", "banana", "orange"], the corresponding indices might be [0, 1, 2], where "apple" is represented by index 0, "banana" by index 1, and "orange" by index 2.


**```dataset```** is an iterable therefore we use a generator function yield_tokens to apply the **```tokenizer```**. The purpose of the generator function **```yield_tokens```** is to yield tokenized texts one at a time. Instead of processing the entire dataset and returning all the tokenized texts in one go, the generator function processes and yields each tokenized text individually as it is requested. The tokenization process is performed lazily, which means the next tokenized text is generated only when needed, saving memory and computational resources.


In [ ]:
def yield_tokens(data_iter):
    for  _,text in data_iter:
        yield tokenizer(text)

In [ ]:
my_iterator = yield_tokens(dataset) 

This creates an iterator called **```my_iterator```** using the generator. To begin the evaluation of the generator and retrieve the values, you can iterate over **```my_iterator```** using a for loop or retrieve values from it using the **```next()```** function.


In [ ]:
next(my_iterator)

We build a vocabulary from the tokenized texts generated by the **```yield_tokens```** generator function, which processes the dataset. The **```build_vocab_from_iterator()```** function constructs the vocabulary, including a special token `unk` to represent out-of-vocabulary words. 

### Out-of-vocabulary (OOV)
When text data is tokenized, there may be words that are not present in the vocabulary because they are rare or unseen during the vocabulary building process. When encountering such OOV words during actual language processing tasks like text generation or language modeling, the model can use the `unk` token to represent them.

For example, if the word "apple" is present in the vocabulary, but "pineapple" is not, "apple" will be used normally in the text, but "pineapple" (being an OOV word) would be replaced by the <unk> token.

By including the <unk> token in the vocabulary, you provide a consistent way to handle out-of-vocabulary words in your language model or other natural language processing tasks.


In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Using **```vocab.get_stoi()```**, you can obtain the entire string-to-index mapping of the vocabulary. This mapping is useful when you want to look up the index of a specific word in the vocabulary, which is essential when converting text data into numerical form for various natural language processing tasks.


In [ ]:
vocab.get_stoi()

Prepare the text processing pipeline with the tokenizer and vocabulary. The text and label pipelines will be used to process the raw data strings from the dataset iterators. 

The function **```text_pipeline```** will tokenize the input text, and **```vocab```** will then be applied to get the token indices. 
The **```label_pipeline```** will ensure that the labels start at zero.


In [ ]:
def text_pipeline(x):
  return vocab(tokenizer(x))

def label_pipeline(x):
   return int(x) - 1

We can apply the functions to each element in the dataset, storing the labels in```label_list```, and the Token Indices in```text_list```. Later on, to retrieve the original sample, we use the list offsets, which indicate the size of each sequence.


In [ ]:
label_list, text_list, offsets = [], [], [0]
for i, (_label, _text) in enumerate(dataset):
    print("iteration",i)   
    print("label:",_label)
    new_label=label_pipeline(_label)
    print("new label:",new_label)
    label_list.append(new_label)
        
    print("text:",_text)
    processed_text=text_pipeline(_text)
    print("processed text:",processed_text)
        
    processed_text = torch.tensor(processed_text, dtype=torch.int64)
    text_list.append(processed_text)
    print("offsets:",processed_text.size(0))
        
    offsets.append(processed_text.size(0))
    print("\n")

We change ```label_list``` to a tensor to work with PyTorch.


In [ ]:
label_list = torch.tensor(label_list, dtype=torch.int64)
label_list

In [ ]:
text_list

We see each sequence in the list ```text_list``` is of a different length, we will flatten the list making processing more efficient.

**Why do we flatten the list?**

By flattening the list and using cumulative offsets, we create a unified, contiguous representation of the entire text. This approach eliminates the need for padding (which essentially involves keeping sentences separate and of equal length, but can be memory-intensive). As a result, the model can process the text more effectively without having to handle individual sequences separately. This approach can lead to faster computations and reduced memory usage.


In [ ]:
 text_list = torch.cat(text_list)
 text_list

Offsets represent the length of each sequence. By applying the`cumsum`function, we can find the position of each sample in```text_list```. Each element in the output will be the cumulative sum of each sample's size, providing the precise index position for each sample in```text_list```.


In [ ]:
print(offsets)
offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
offsets

---


## Embedding

### Embeddings in PyTorch 

We can represent each word as a special kind of number, which we call a "word vector" or "embedding." These word vectors are like coordinates in a lower-dimensional space. Let's say we want to represent three words: "cat," "dog," and "bird." We can create word vectors for each of them as follows:

Word "cat": $\mathbf{e}_{\text{cat}} = [0.8, 0.5]$

Word "dog": $\mathbf{e}_{\text{dog}} = [0.3, -0.2]$

Word "bird": $\mathbf{e}_{\text{bird}} = [-0.6, 0.9]$

The magic of these word vectors is that words with similar meanings or used in similar contexts end up closer to each other in this lower-dimensional space. In our example, if we visualize these points, we might see that "cat" and "dog" are closer to each other than either is to "bird." This closeness helps the computer understand the relationships between words, which is useful for various machine learning tasks.

![2d](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-GPXX0Y15EN/Screenshot%202023-07-31%20at%208.26.00%20AM.png)


**```nn.Embedding(vocab_size, embedding_dim)```** is a PyTorch layer used in natural language processing. It converts words (represented as integers) into compact and meaningful vectors of a specified size. The `vocab_size` is the number of unique words, and `embedding_dim` is the size of the resulting word vectors. For example, if `vocab_size=3` and `embedding_dim=2`, each word will be transformed into a 2-dimensional vector.


In [ ]:
embedding_dim=4

vocab_size=len(vocab)
print(vocab_size)

In [ ]:
embedding = nn.Embedding(vocab_size, embedding_dim)

 Embedding are randomly initialized, we will set them to our randomly initialized values for later.


In [ ]:
wights_=torch.randn(22,4)
embedding.weight.data=wights_

The random embedding values are stored in tokens.


In [ ]:
for word in dataset[1][1].split():
    print("word:",word)
    print("Token Indices",text_pipeline(word))
    print("embedding",embedding(torch.tensor(text_pipeline(word))))

### Embedding Bag

Now, a sentence can have different lengths (more words or fewer words), and this can be a problem when we want to use this information in a computer program. So, to solve this problem, we use "embedding bag". **Embedding bag** that takes all the word embeddings in a sentence and combines them to create a single fixed-length representation that summarizes the overall meaning of the whole sentence. It does this by averaging the word embeddings together.

An `embedding_bag` in PyTorch takes dense vectors of a specified size (embedding_dim), similar to the `Embedding` layer. However, the key difference is that `embedding_bag` performs a pooling operation (like averaging) on the input embeddings to generate a single fixed-length vector, whereas the `Embedding` layer maps each input to a unique vector from a lookup table.


In [ ]:
embedding_bag=nn.EmbeddingBag(vocab_size, embedding_dim)

The parameters of embedding bag are randomly initialized; we will set them to our randomly initialized values so we can compare them to the Embedding:


In [ ]:
embedding_bag.weight.data=wights_

The input consists of token indices, where 'offsets' determine the starting index position. If we only have one sequence, the offset value is 0. The code below showcases this.


In [ ]:
embedding_bag(torch.tensor(text_pipeline(dataset[1][1])),offsets=torch.tensor([0]))

We can show that the output of **`embedding_bag`** is just the average of embeddings.


In [ ]:
for y,sentence in dataset:
    print("sentence:",sentence)
    my_tokenizes=tokenizer(sentence)
    print("tokens:",my_tokenizes)
    token_indices=vocab(my_tokenizes)
    my_embeddings=embedding(torch.tensor(token_indices).reshape(-1))
    print("my embeddings \n",my_embeddings)
    print("mean embeddings:",my_embeddings.mean(0))
    my_embeddings_bag=embedding_bag(torch.tensor(token_indices),torch.tensor([0]))
    print("embeddings bag:",my_embeddings_bag)
    print("\n")

By inputting multiple sequences from `text_list`, the `offsets` will provide precise location information for each sequence. The resulting output will consist of an embedding bag representing each individual sequence.


In [ ]:
embedding_bag( text_list ,offsets=offsets)

If you inspect, the `embedding_bag` collection will be the same for each generated embeddings bag.

Now after going through all of this. The gif below gives you an overview.

![nlp](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-GPXX0Y15EN/NLP.gif)


---


### Import bank dataset

Load the AG_NEWS dataset for the train split and split it into input text and corresponding labels:


In [ ]:
train_iter= AG_NEWS(split="train")

The AG_NEWS dataset in torchtext does not support direct indexing like a list or tuple. It is not a random access dataset but rather an iterable dataset that needs to be used with an iterator. This approach is more effective for text data.


In [ ]:
y,text= next(iter(train_iter ))
print(y,text)

We can find the label of the sample.


In [ ]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
ag_news_label[y]

We can also use the dataset to find all the classes.


In [ ]:
num_class = len(set([label for (label, text) in train_iter ]))
num_class 

We can build the vocabulary as before, just using the AG dataset to obtain token indices


In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Here are some token indices:


In [ ]:
vocab(["age","hello"])

### Dataset 


We can convert the dataset into map-style datasets and then perform a random split to create separate training and validation datasets. The training dataset will contain 95% of the samples, while the validation dataset will contain the remaining 5%. These datasets can be used for training and evaluating a machine learning model for text classification on the AG_NEWS dataset.


In [ ]:
# Split the dataset into training and testing iterators.
train_iter, test_iter = AG_NEWS()

# Convert the training and testing iterators to map-style datasets.
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Determine the number of samples to be used for training and validation (5% for validation).
num_train = int(len(train_dataset) * 0.95)

# Randomly split the training dataset into training and validation datasets using `random_split`.
# The training dataset will contain 95% of the samples, and the validation dataset will contain the remaining 5%.
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

The code checks if a CUDA-compatible GPU is available in the system using PyTorch, a popular deep learning framework. If a GPU is available, it assigns the device variable to "cuda" (which stands for CUDA, the parallel computing platform and application programming interface model developed by NVIDIA). If a GPU is not available, it assigns the device variable to "cpu" (which means the code will run on the CPU instead).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Data Loader


In PyTorch, the **`collate_fn`** function is used in conjunction with data loaders to customize the way batches are created from individual samples. The provided code defines a `collate_batch` function in PyTorch, which is used with data loaders to customize batch creation from individual samples. It processes a batch of data, including labels and text sequences. It applies the `label_pipeline` and `text_pipeline` functions to preprocess the labels and texts, respectively. The processed data is then converted into PyTorch tensors and returned as a tuple containing the label tensor, text tensor, and offsets tensor representing the starting positions of each text sequence in the combined tensor. The function also ensures that the returned tensors are moved to the specified device (e.g., GPU) for efficient computation.


In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

We convert the dataset objects to a data loader by applying the collate function.


In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

We can see the output sequence when we have the label, text, and offsets for each batch.


In [ ]:
label, text, offsets=next(iter(valid_dataloader ))
label, text, offsets

### Neural Network


We have created a neural network for a text classification model using an `EmbeddingBag` layer, followed by a softmax output layer. Additionally, we have initialized the model using a specific method.


In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

We have created the model, and the embedding dimension size is a free parameter.


In [ ]:
emsize=64

We need the vocabulary size to determine the number of embeddings.


In [ ]:
vocab_size=len(vocab)
vocab_size

We have also determined the number of classes for the output layer.


In [ ]:
num_class 

Creating the model:


In [ ]:
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
model

The code line `predicted_label=model(text, offsets)` is used to obtain predicted labels from a machine learning model for a given input text and its corresponding offsets. The `model` is the machine learning model being used for text classification or similar tasks.


In [ ]:
predicted_label=model(text, offsets)

We verify the output shape of our model. In this case, the model is trained with a mini-batch size of 64 samples. The output layer of the model produces 4 logits for each neuron, corresponding to the four classes in the classification task. We can also create a function to find the accuracy given a dataset.


In [ ]:
predicted_label.shape

Function **`predict`** takes in a text and a text pipeline, which preprocesses the text for machine learning. It uses a pre-trained model to predict the label of the text for text classification on the AG_NEWS dataset. The function returns the predicted label as a result.


In [ ]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return ag_news_label[output.argmax(1).item() + 1]

In [ ]:
predict("I like sports",text_pipeline )

We create a function to evaluate the model's accuracy on a dataset.


In [ ]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count= 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)

            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

We proceeded to evaluate the model, and upon observation, we found that its performance is no better than average. This outcome is expected, considering that the model has not undergone any training yet.


In [ ]:
evaluate(test_dataloader)

---


## Train the Model

We set the learning rate (LR) to 0.1, which determines the step size at which the optimizer updates the model's parameters during training. The CrossEntropyLoss criterion is used to calculate the loss between the model's predicted outputs and the ground truth labels. This loss function is commonly employed for multi-class classification tasks.

The chosen optimizer is Stochastic Gradient Descent (SGD), which optimizes the model's parameters based on the computed gradients with respect to the loss function. The SGD optimizer uses the specified learning rate to control the size of the weight updates.

Additionally, a learning rate scheduler is defined using StepLR. This scheduler adjusts the learning rate during training, reducing it by a factor (gamma) of 0.1 after every epoch (step) to improve convergence and fine-tune the model's performance. These components together form the essential setup for training a neural network using the specified learning rate, loss criterion, optimizer, and learning rate scheduler.


In [ ]:
LR=0.1

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

Training the model, which should take about 20 minutes.


In [ ]:
EPOCHS = 10
cum_loss_list=[]
acc_epoch=[]
acc_old=0

for epoch in tqdm(range(1, EPOCHS + 1)):
    model.train()
    cum_loss=0
    for idx, (label, text, offsets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        cum_loss+=loss.item()

    cum_loss_list.append(cum_loss)
    accu_val = evaluate(valid_dataloader)
    acc_epoch.append(accu_val)

    if accu_val > acc_old:
      acc_old= accu_val
      torch.save(model.state_dict(), 'my_model.pth')

In [ ]:
plot(cum_loss_list,acc_epoch)

In [ ]:
evaluate(test_dataloader)

This code snippet provides a summary for generating a 3D t-SNE visualization of embeddings using Plotly. It demonstrates how words that are similar to each other are positioned closer together.


In [ ]:
# Get the first batch from the validation data
batch = next(iter(valid_dataloader))

# Extract the text and offsets from the batch
label, text, offsets = batch

# Send the data to the device (GPU if available)
text = text.to(device)
offsets = offsets.to(device)

# Get the embeddings bag output for the batch
embedded = model.embedding(text, offsets)

# Convert the embeddings tensor to a numpy array
embeddings_numpy = embedded.detach().cpu().numpy()

# Perform t-SNE on the embeddings to reduce their dimensionality to 3D.
X_embedded_3d = TSNE(n_components=3).fit_transform(embeddings_numpy)

# Create a 3D scatter plot using Plotly
trace = go.Scatter3d(
    x=X_embedded_3d[:, 0],
    y=X_embedded_3d[:, 1],
    z=X_embedded_3d[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=label.numpy(),  # Use label information for color
        colorscale='Viridis',  # Choose a colorscale
        opacity=0.8
    )
)

layout = go.Layout(title="3D t-SNE Visualization of Embeddings",
                   scene=dict(xaxis_title='Dimension 1',
                              yaxis_title='Dimension 2',
                              zaxis_title='Dimension 3'))

fig = go.Figure(data=[trace], layout=layout)
fig.show()

We can make a prediction on the following article using the function **`predict`**.


In [ ]:
article="""Canada navigated a stiff test against the Republic of Ireland on a rain soaked evening in Perth, coming from behind to claim a vital 2-1 victory at the Women’s World Cup.
Katie McCabe opened the scoring with an incredible Olimpico goal – scoring straight from a corner kick – as her corner flew straight over the despairing Canada goalkeeper Kailen Sheridan at Perth Rectangular Stadium in Australia.
Just when Ireland thought it had safely navigated itself to half time with a lead, Megan Connolly failed to get a clean connection on a clearance with the resulting contact squirming into her own net to level the score.
Minutes into the second half, Adriana Leon completed the turnaround for the Olympic champion, slotting home from the edge of the area to seal the three points."""

This markdown content generates a styled box with light gray background and padding. It contains an `<h3>` header displaying the content of the `article` variable, and an `<h4>` header indicating the predicted category of the news article which is provided by the `result` variable. The placeholders `{article}` and `{result}` will be dynamically replaced with actual values when this markdown is rendered.


In [ ]:
result = predict(article, text_pipeline)

markdown_content = f'''
<div style="background-color: lightgray; padding: 10px;">
    <h3>{article}</h3>
    <h4>The category of the news article: {result}</h4>
</div>
'''

md(markdown_content)